In [4]:
import pandas as pd
import numpy as np

In [5]:
def mergeData():
    monday = pd.read_csv(
        "../Data/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")

    # 剔除第一行属性特征名称
    monday = monday.drop([0])
    friday1 = pd.read_csv(
        "../Data/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
    friday1 = friday1.drop([0])
    friday2 = pd.read_csv(
        "../Data/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
    friday2 = friday2.drop([0])
    friday3 = pd.read_csv(
        "../Data/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
    friday3 = friday3.drop([0])
    thursday1 = pd.read_csv(
        "../Data/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
    thursday1 = thursday1.drop([0])
    thursday2 = pd.read_csv(
        "../Data/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
    thursday2 = thursday2.drop([0])
    tuesday = pd.read_csv(
        "../Data/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
    tuesday = tuesday.drop([0])
    wednesday = pd.read_csv(
        "../Data/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")
    wednesday = wednesday.drop([0])
    return pd.concat([monday, friday1, friday2, friday3, thursday1, thursday2, tuesday, wednesday], axis=0)

In [6]:
raw_data = mergeData()
raw_data.shape

(2830735, 79)

In [7]:
# 去除异常值，删除所有包含NaN的行
# 去除Infinity值
raw_data = raw_data.replace([np.inf, -np.inf], np.nan)
raw_data = raw_data.dropna()
raw_data.shape

(2827868, 79)

### 去除所有异常值后的数据

In [8]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2827868 entries, 1 to 692702
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max               

In [9]:
# 获取标签数据，并统计
last_column_index = raw_data.shape[1] - 1
label = raw_data.iloc[:, last_column_index]
label.value_counts()

 Label
BENIGN                        2271312
DoS Hulk                       230124
PortScan                       158804
DDoS                           128025
DoS GoldenEye                   10293
FTP-Patator                      7935
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1956
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

### 整个数据集相当不平衡，正常数据非常大，而攻击流量却相当少

### 数据重采样，使得正负样本均衡，将每个数据扩充到5000个以上
按最后一列的值进行分组

In [ ]:
grouped = raw_data.groupby(raw_data.iloc[:, last_column_index])
grouped.size()

 Label
BENIGN                        2271312
Bot                              1956
DDoS                           128025
DoS GoldenEye                   10293
DoS Hulk                       230124
DoS Slowhttptest                 5499
DoS slowloris                    5796
FTP-Patator                      7935
Heartbleed                         11
Infiltration                       36
PortScan                       158804
SSH-Patator                      5897
Web Attack � Brute Force         1507
Web Attack � Sql Injection         21
Web Attack � XSS                  652
dtype: int64

In [11]:
# 重采样
resampled_data = grouped.apply(lambda x: x.sample(5000, replace=True))
resampled_data = resampled_data.reset_index(drop=True)
resampled_data.shape, resampled_data.iloc[:, last_column_index].value_counts()

((75000, 79),
  Label
 BENIGN                        5000
 Bot                           5000
 DDoS                          5000
 DoS GoldenEye                 5000
 DoS Hulk                      5000
 DoS Slowhttptest              5000
 DoS slowloris                 5000
 FTP-Patator                   5000
 Heartbleed                    5000
 Infiltration                  5000
 PortScan                      5000
 SSH-Patator                   5000
 Web Attack � Brute Force      5000
 Web Attack � Sql Injection    5000
 Web Attack � XSS              5000
 Name: count, dtype: int64)

In [12]:
data = resampled_data.iloc[:, 0:last_column_index]
label = resampled_data.iloc[:, last_column_index]

In [13]:
data.info(), label.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 78 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0    Destination Port             75000 non-null  int64  
 1    Flow Duration                75000 non-null  int64  
 2    Total Fwd Packets            75000 non-null  int64  
 3    Total Backward Packets       75000 non-null  int64  
 4   Total Length of Fwd Packets   75000 non-null  int64  
 5    Total Length of Bwd Packets  75000 non-null  int64  
 6    Fwd Packet Length Max        75000 non-null  int64  
 7    Fwd Packet Length Min        75000 non-null  int64  
 8    Fwd Packet Length Mean       75000 non-null  float64
 9    Fwd Packet Length Std        75000 non-null  float64
 10  Bwd Packet Length Max         75000 non-null  int64  
 11   Bwd Packet Length Min        75000 non-null  int64  
 12   Bwd Packet Length Mean       75000 non-null  float64
 13   

(None,
  Label
 BENIGN                        5000
 Bot                           5000
 DDoS                          5000
 DoS GoldenEye                 5000
 DoS Hulk                      5000
 DoS Slowhttptest              5000
 DoS slowloris                 5000
 FTP-Patator                   5000
 Heartbleed                    5000
 Infiltration                  5000
 PortScan                      5000
 SSH-Patator                   5000
 Web Attack � Brute Force      5000
 Web Attack � Sql Injection    5000
 Web Attack � XSS              5000
 Name: count, dtype: int64)

In [14]:
# 标签化
labels_encoded, uniques = pd.factorize(label)
labels_encoded, uniques

(array([ 0,  0,  0, ..., 14, 14, 14], dtype=int64),
 Index(['BENIGN', 'Bot', 'DDoS', 'DoS GoldenEye', 'DoS Hulk',
        'DoS Slowhttptest', 'DoS slowloris', 'FTP-Patator', 'Heartbleed',
        'Infiltration', 'PortScan', 'SSH-Patator', 'Web Attack � Brute Force',
        'Web Attack � Sql Injection', 'Web Attack � XSS'],
       dtype='object'))

In [15]:
import os

labels_encoded = pd.DataFrame(labels_encoded)
unique = pd.Series(uniques)

In [17]:
if not os.path.exists("../Data/processed_data"):
    os.mkdir("../Data/processed_data")
data.to_csv("../Data/processed_data/data.csv", index=False)
labels_encoded.to_csv("../Data/processed_data/labels.csv", index=False)

In [18]:
# 以字典形式保存标签
import json
with open("../Data/processed_data/unique.json", "w") as f:
    result = {
        x : y for x, y in zip(range(len(unique)), unique)
    }
    json.dump(result, f)